# Youtube Data API

In [1]:
!pip install google-api-python-client

   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ---------------------- ----------------- 7.3/13.1 MB 37.6 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 37.3 MB/s eta 0:00:00


In [4]:
import json
from googleapiclient.discovery import build

In [9]:
DEVELOPER_KEY = json.load(open('./keys/youtube_key.json'))['api_key']
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(
    YOUTUBE_API_SERVICE_NAME,
    YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY
)


In [16]:
# Get Channel object

# 1. First find Channel ID by Channel Name

request = youtube.search().list(
        part="snippet",
        maxResults=5,
        q="Barbascura eXtra",
        type="channel"
    )

response = request.execute()

channelId = response['items'][0]['id']['channelId']

channelId

'UCHi6Q3Z-5oJUC691WLlSntA'

In [17]:
# 2. Then fetch the information we need about it

request = youtube.channels().list(
    part="snippet,contentDetails",
    id= channelId
    )
response = request.execute()
uploaded_playlist = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
uploaded_playlist

'UUHi6Q3Z-5oJUC691WLlSntA'

In [31]:
channel_videos : list = list()

page_token = None

while True:
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            maxResults=50,
            playlistId=uploaded_playlist,
            pageToken = page_token
    )

    response = request.execute()

    channel_videos.extend(
        v['contentDetails']['videoId']
        for v in response['items']
    )

    if response.get('nextPageToken', None) is None:
        break

    page_token = response['nextPageToken']

len(channel_videos)

378

In [49]:
# Get Comments for Every Video
commentators : set = set()

for video_id in channel_videos[:10]:

    page_token = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken = page_token
        )

        response = request.execute()

        commentators.update(
            ct['snippet']['topLevelComment']['snippet']['authorDisplayName']
            for ct in response['items']
        )

        if response.get('nextPageToken', None) is None or True:
            break

        page_token = response['nextPageToken']



In [50]:
len(commentators), commentators

(794,
 {'@-GTN-',
  '@-_Ken_-HSF',
  '@-abes8421',
  '@2307andrea',
  '@36Otrip',
  '@5nak389',
  '@7788abel',
  '@90salta90',
  '@93bovo93',
  '@95annachiara',
  '@ARCANO-p1b',
  '@Abigaildidonato9748',
  '@AdelinaMorenos',
  '@Aesthetic_aespa_2001',
  '@AlFaroBeer',
  '@AlSc-p3t',
  '@AlbertoAndreani-l9k',
  '@AlbertoPaolini-cg5oo',
  '@Alex89rm_',
  '@Alexis.350',
  '@AliceLiso-rn2xw',
  '@Anam404.',
  '@AndreAIXIDOR89',
  '@Andrea-b2b9b',
  '@AndreaLunaXD',
  '@Andreas_Æ-Eagle',
  '@Anglachel16',
  '@Ankaaalpha',
  '@AnonimoKING-ho5id',
  '@Anpìbu',
  '@Ash-Nox',
  '@Ashlaracnes',
  '@Aura24680',
  '@Aury226',
  '@BarbascuraEXtra',
  '@BenJack85',
  '@BertolliRobusto',
  '@BlackWorld91',
  '@BladeDMC',
  '@BlessedUFC',
  '@Bollazza',
  '@Bonini000',
  '@BorisRomagna',
  '@Bruno-zg4cx',
  '@Butterfly.effect.6',
  '@CHARLIE_pickle',
  '@CRSTN1312',
  '@CaciaraRap',
  '@Callmemissnaive',
  '@CarmineBentivedo',
  '@Ceros2112',
  '@CherishHaze666',
  '@Childofposeidon35',
  '@ChriBacchi